# BEAST Analysis Notebook

---

# 0. SETUP

In [1]:
import os
import pandas as pd
import seaborn as sns
from Bio import Phylo
from functions import *

## Paths

In [2]:
log_dir  = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade"
tree_dir = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock"
metadata_path = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/iqtree/all/chromosome/full/filter5/filter-taxa/metadata.tsv"
beast_dir = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade"

auspice_config_path = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json"
out_path_colors = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/augur/all/chromosome/full/filter5/colors.tsv"
out_path_latlon = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/augur/all/chromosome/full/filter5/latlon.tsv"

## Variables

In [3]:
# ------------------------------------------
BRANCH_LIST = {
    "1.ORI" : ["1.ORI1", "1.ORI2", "1.ORI3"],
    "1.IN": ["1.IN1","1.IN2","1.IN3"],  
    "1.ANT": ["1.ANT1"], 
    "1.PRE" : ["1.PRE0","1.PRE1", "1.PRE2", "1.PRE3"],
    "2.MED": ["2.MED0", "2.MED1","2.MED2","2.MED3" ],      
    "2.ANT": ["2.ANT1","2.ANT2","2.ANT3" ],    
    "4.ANT": ["4.ANT1" ],       
    "3.ANT": ["3.ANT1", "3.ANT2" ],  
    "0.ANT": ["0.ANT1", "0.ANT2","0.ANT3","0.ANT5"],         
    "0.ANT4" : ["0.ANT4"], 
    "0.PE": ["0.PE2", "0.PE4m", "0.PE4m", "0.PE4t", "0.PE4a", "0.PE5", "0.PE7", "0.PE8", "0.PE10"],   
    "0.PRE": ["0.PRE1", "0.PRE2"],        
}

NUM_STATES = 10

NO_DATA_CHAR = "NA"
JSON_INDENT=2

---

# 1. IMPORT

## Metadata

In [4]:
metadata_df = pd.read_csv(metadata_path, sep='\t')
metadata_df.set_index(metadata_df.columns[0], inplace=True)
metadata_df.fillna(NO_DATA_CHAR, inplace=True)

display(metadata_df)

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,biosample_accession,biosample_comment,branch_number,continent,date_mean,date_bp_mean,date_err,lat,lon,host_human
sample,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.783730,-100.445882,38.7252,-105.608,Orientalis,...,SAMEA1705942,KEEP: Assembly Modern Reference,1,North America,1992.0,29.0,0.0,38.725178,-105.607716,Human
GCA_009909635.1_ASM990963v1_genomic,9_10,1923.0,-98,Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Medievalis,...,SAMN13632815,KEEP: Assembly Modern,2,Europe,1923.0,98.0,0.0,47.622245,40.795794,Human
GCA_009669545.1_ASM966954v1_genomic,42126,2006.0,-15,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722925,KEEP: Assembly Modern,0,Asia,2006.0,15.0,0.0,42.480495,85.463346,Non-Human
GCA_009669555.1_ASM966955v1_genomic,42123,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722924,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human
GCA_009669565.1_ASM966956v1_genomic,42118,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,...,SAMN07722923,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMEA7313243_45,Azov38,[1400:1700],[-621:-321],Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Second Pandemic,...,SAMEA7313243_45,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,47.622245,40.795794,Human
SAMEA7313246_49,Gdansk8,[1400:1700],[-621:-321],Poland,Pomeranian Voivodeship,52.215933,19.134422,54.2456,18.1099,Second Pandemic,...,SAMEA7313246_49,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,54.245560,18.109900,Human
SAMEA6651390,AGU010,[1435:1477],[-586:-544],Lithuania,Vilnius County,55.350000,23.750000,54.8227,25.2495,Second Pandemic,...,SAMEA6651390,KEEP: SRA Ancient,1,Europe,1456.0,565.0,21.0,54.822692,25.249534,Human


## Most Recent Sampling Date

In [5]:
out_path = os.path.join(beast_dir, "most_recent_sampling_dates.tsv")
mrsd_dict = {}

with open(out_path, "w") as outfile:
    for branch in BRANCH_LIST:
        branch_df = metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])]
        max_date = max(branch_df["date_mean"])
        outfile.write("{}\t{}\n".format(branch, max_date))
        mrsd_dict[branch] = max_date

## Colors

In [6]:
colors_dict = {}

colors_df = pd.read_csv(out_path_colors, sep='\t')
colors_df.columns = ["state", "value", "color"]

for state in set(colors_df["state"]):
    state_df = colors_df[colors_df["state"] == state]    
    colors_dict[state] = {}
    
    for value,color in zip(state_df["value"], state_df["color"]):
        colors_dict[state][value] = color

print(colors_dict)

{'province': {'Krasnoyarsk Krai': '#8000ff', 'Panevezys County': '#7b07ff', 'Pärnu maakond': '#760eff', 'Irkutsk Oblast': '#7215ff', 'Bavaria': '#6d1dff', 'Altai Krai': '#6924fe', 'Qinghai': '#642bfe', 'Republic of Dagestan': '#6032fe', 'Goranboy District': '#5b39fd', 'Syunik Province': '#5740fd', 'Shirak Province': '#5247fc', 'Gegharkunik Province': '#4d4dfc', 'Samtskhe-Javakheti': '#4954fb', 'Shahbuz Rayon': '#445bfb', 'Samara Oblast': '#4062fa', 'Sečuán': '#3b68f9', 'Bayankhongor': '#376ff9', 'Inner Mongolia': '#3275f8', 'Sughd Province': '#2e7bf7', 'Talas Region': '#2982f6', 'Altai Republic': '#2488f5', 'Govi-Altai': '#208ef4', 'Övörkhangai': '#1b94f3', 'Ömnögovi': '#1799f2', 'Bayan-Ölgii': '#129ff1', 'Khovd': '#0ea5ef', 'Xinjiang': '#09aaee', 'Issyk-Kul Region': '#05afed', 'East of England': '#00b4ec', 'Valencia Community': '#05b9ea', 'Centre-Loire Valley': '#09bee9', 'Osh Region': '#0ec3e7', 'Naryn Region': '#12c7e6', 'Gansu': '#17cce4', 'Hovsgel': '#1bd0e3', 'Zavkhan Province': 

## Tree Files

In [7]:
# Construct a dictionary to hold the trees
tree_dict = {}

NEXUS_EXCLUDE_LINES = ["#NEXUS"]

for branch in BRANCH_LIST:
    tree_dict[branch] = {} 
    for filename in os.listdir(tree_dir):
        if not filename.endswith(".tre"): continue
        filepath = os.path.join(tree_dir, filename)
        if branch in filename:            
            # Add tree files to dict
            tree_dict[branch]["tree_file_raw"] = filepath 
            tree_dict[branch]["tree_file_edit"] = os.path.join(tree_dir, branch + ".nex")
            tree_dict[branch]["sample_rename"] = {}
            
            # Read in raw tree to deal with dashes
            with open(tree_dict[branch]["tree_file_raw"],  "r") as infile:                    
                with open(tree_dict[branch]["tree_file_edit"], "w") as outfile:           
                    raw_tree = infile.read()
                    # Remove quotations if they exist
                    raw_tree = raw_tree.replace("'","")
                    
                    # Split into lines to iterate over
                    raw_tree_lines = raw_tree.split("\n")
                    
                    # By default don't parse a line for dashes
                    taxa_line = False
                    
                    for line in raw_tree_lines: 
                                
                        if "TREE" not in line and "-" in line:
                            
                            if len(line.split(" ")) == 1:
                                name_dashes = line.strip()
                                name_no_dashes = name_dashes.replace("-","_")
                                tree_dict[branch]["sample_rename"][name_no_dashes] = name_dashes
                            line = line.replace("-","_")
                                
                        outfile.write(line + "\n")
            
            # Read in edited tree
            trees = Phylo.parse(tree_dict[branch]["tree_file_edit"], "nexus")
            print(trees)
            # There should be only 1 tree
            for t in trees:
                tree_dict[branch]["tree"] = t
                tree_dict[branch]["tree"].ladderize(reverse=False)
                break

            # Rename sample names back to with dashes
            for c in tree_dict[branch]["tree"].find_clades():
                if c.name in tree_dict[branch]["sample_rename"]:
                    orig_name = c.name
                    c.name = tree_dict[branch]["sample_rename"][c.name]
                    print("Rename:", orig_name, c.name)
                    
                # Strip the date suffix
                if c.name:
                    c.name = "_".join(c.name.split("_")[0:-1])
                    
    break
    if branch == "1.IN":
        break

<generator object parse at 0x7f2c9c5ca750>
Rename: GCA_000324805.2_EV76_CN_genomic_90 GCA_000324805.2_EV76-CN_genomic_90
Rename: GCA_000986995.1_YPES001_SEQ_2_ASM_1_genomic_22 GCA_000986995.1_YPES001-SEQ-2-ASM-1_genomic_22


## Add Tree Data to Dataframe

In [8]:
# Initialize new columns

metadata_df["node_type"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["branch_length"] = [NO_DATA_CHAR] * len(metadata_df)
metadata_df["timetree_num_date_confidence"] = [[0,0]] * len(metadata_df)

init_data = {col: [NO_DATA_CHAR] for col in metadata_df.columns}

for branch in BRANCH_LIST:
    tree = tree_dict[branch]["tree"]

    df = copy.deepcopy(metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])])
    
    node_i = 0
    
    # Get comment headers for df
    root_comment_dict = parse_comment(tree.root.comment)
    for parameter in root_comment_dict:
        df[parameter] = [NO_DATA_CHAR] * len(df)
    
    for c in tree.find_clades():

        # Rename internal nodes
        if not c.name:
            c.name = "NODE{}".format(node_i)
            node_i += 1

        # Initialize metadata for internal nodes
        if "NODE" in c.name:
            init_data["sample"] = c.name
            data_row = pd.DataFrame(init_data)
            data_row.set_index("sample", inplace=True)
            df = df.append(data_row)
            
        # Set node type
        if "NODE" in c.name:
            node_type = "internal"                    
        else:
            node_type = "terminal"
           
        # Parse comments
        comment_dict = parse_comment(c.comment)
        for parameter,val in comment_dict.items():
            df.at[c.name, parameter] = val
            
        # Get calendar date for node
        calendar_date = mrsd_dict[branch] - float(comment_dict["height"])
        df.at[c.name, "timetree_num_date"] = calendar_date
        
        
        # Convert
        height = comment_dict["height"]
        df.at[c.name, "height"] = float(height)
        if "height_95%_HPD" in comment_dict:
            height_95_hpd = comment_dict["height_95%_HPD"]
            height_95_hpd_list = [float(h) for h in height_95_hpd.strip("{}").split(",")]
            df.at[c.name, "timetree_num_date_confidence"] = height_95_hpd_list
        
        # Add data
        df.at[c.name, "node_type"] = node_type
        df.at[c.name, "branch_length"] = c.branch_length

        
        
    # Replace NAs for inconsistent parameters
    df.fillna(NO_DATA_CHAR, inplace=True)
    tree_dict[branch]["df"] = copy.deepcopy(df)

    
    display(tree_dict[branch]["df"])
    break


,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,...,height_range,height,timetree_num_date,length_range,rate_95%_HPD,length_95%_HPD,rate_range,rate,rate_median,length_median
sample,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.7837,-100.446,38.7252,-105.608,Orientalis,...,"{23.99999999999997,24.00000000000003}",24.000000,1992.000000,"{0.7875393122285885,42.88322836127834}","{5.927714716500102E-8,9.377226168027601E-7}","{1.8427926537330457,31.191320981608243}","{4.558602518220217E-8,5.030931524742551E-6}",3.5537168881951186E-7,2.406871030661851E-7,15.627963378354789
GCA_000834775.1_ASM83477v1_genomic,Dodson,1967.0,-54,United States of America,Arizona,39.7837,-100.446,34.3953,-111.763,Orientalis,...,"{48.99999999999994,49.00000000000006}",49.000000,1967.000000,"{0.2888043881030171,31.897590286999048}","{3.797620557463332E-8,8.218425612961558E-7}","{0.5471598315006432,15.847060277531043}","{3.429741540262673E-8,5.587063400379781E-6}",3.123540237046625E-7,2.11059520244398E-7,6.8709840507756965
GCA_000834335.1_ASM83433v1_genomic,Shasta,1954.0,-67,United States of America,California,39.7837,-100.446,36.7015,-118.756,Orientalis,...,"{61.99999999999994,62.00000000000006}",62.000000,1954.000000,"{0.05748774760200348,28.285740849649073}","{9.133766746749261E-9,6.323190957865774E-7}","{0.17495681504591687,12.737728729624394}","{8.854592434294595E-9,1.0406855638781851E-5}",2.1221803830116104E-7,1.3435424636585818E-7,4.22066337192107
GCA_000169635.1_ASM16963v1_genomic,MG05-1020,2005.0,-16,Madagascar,NA,-18.925,46.4416,NA,NA,Orientalis,...,"{10.999999999999886,11.000000000000114}",11.000000,2005.000000,"{8.752165669176431,295.5000109399901}","{1.233569124508418E-8,2.3007345037878172E-7}","{10.589101358759361,123.010848907607}","{2.4954871920903633E-9,6.269225969490349E-7}",1.0267172296033236E-7,9.170997173986732E-8,31.207977460374813
GCA_000170275.1_ASM17027v1_genomic,F1991016,1991.0,-30,China,Yunnan,35.0001,105,25,102,Orientalis,...,"{24.999999999999915,25.000000000000114}",25.000000,1991.000000,"{0.2631175955287688,44.1379271393413}","{1.335420651988494E-8,1.744799312573441E-7}","{4.1125205412269175,25.461413283591654}","{7.906603889093561E-9,8.136140381553714E-6}",8.033665431846963E-8,6.45564613655224E-8,14.682564786518267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE111,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,"{7.792388509136714,21.612246615199865}",11.758628,2004.241372,"{7.716029389008838E-4,11.312580561638354}","{1.9839302934854463E-9,3.12708553465188E-7}","{7.716029389008838E-4,4.39721893027677}","{1.3911910216568945E-9,4.122322400208003E-6}",1.0165048023179178E-7,5.489378888559102E-8,0.876279012156477
NODE112,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,"{6.223285280518212,20.760408533631022}",9.000250,2006.999750,"{0.014623758088442074,13.240021205452297}","{3.99933677937127E-9,3.777893049181355E-7}","{0.04462381365286738,6.681793251825638}","{1.2441548987194403E-10,2.184978370128529E-6}",1.3112765448279205E-7,8.569340550062568E-8,2.584203274463891
NODE113,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,"{6.250713139639117,20.59096663766985}",10.153429,2005.846571,"{0.013053801168137369,14.049681722628108}","{4.988541760516177E-9,5.009242745920154E-7}","{0.0364229231711537,5.210916537824495}","{3.2679056126377386E-9,3.673718936276794E-6}",1.6531290969460705E-7,1.0063395444008382E-7,1.7175801021600883


## Manual Editing

In [9]:
"""for branch in tree_dict:
    
    
    for rec in tree_dict[branch]["df"].iterrows():
        tree_dict[branch]["df"]["timetree_num_date_confidence"] = [0,0] * len(metadata_df)
        
        node = rec[0]
        # Fix the most recent sample having NA for this
        height_95_hpd = rec[1]["height_95%_HPD"]
        if height_95_hpd == NO_DATA_CHAR:
            tree_dict[branch]["df"].at[node, "height_95%_HPD" ] = "{0,0}"
            

        # Convert type
        tree_dict[branch]["df"].at[node, "height"] = float(tree_dict[branch]["df"]["height"][node])
        
    display(tree_dict[branch]["df"])"""

'for branch in tree_dict:\n    \n    \n    for rec in tree_dict[branch]["df"].iterrows():\n        tree_dict[branch]["df"]["timetree_num_date_confidence"] = [0,0] * len(metadata_df)\n        \n        node = rec[0]\n        # Fix the most recent sample having NA for this\n        height_95_hpd = rec[1]["height_95%_HPD"]\n        if height_95_hpd == NO_DATA_CHAR:\n            tree_dict[branch]["df"].at[node, "height_95%_HPD" ] = "{0,0}"\n            \n\n        # Convert type\n        tree_dict[branch]["df"].at[node, "height"] = float(tree_dict[branch]["df"]["height"][node])\n        \n    display(tree_dict[branch]["df"])'

## Reduced Dataframe for Auspice

In [10]:
for branch in tree_dict:
    
    columns = [
        # Node type is mandatory
        "node_type",        
        # Draw Divergence Tree
        "branch_length",
        # Draw Time Tree
        "timetree_num_date",
        "timetree_num_date_confidence",        
    ]
    
    """auspice_df = copy.copy(tree_dict[branch]["df"][columns])
    auspice_df["timetree_num_date_confidence"] = [[0, 0]] * len(auspice_df)

    # Edit df
    for rec in auspice_df.iterrows():
        date_range = rec[1]["height_95%_HPD"]
        timetree_num_date_confidence = [float(d) for d in date_range.strip("{}").split(",")]
        auspice_df.at[rec[0],"timetree_num_date_confidence"] = timetree_num_date_confidence
    
    auspice_df.rename(columns={"height": "timetree_num_date"}, inplace=True)
    auspice_df.drop(columns="height_95%_HPD", inplace=True)"""

    tree_dict[branch]["auspice_df"] = auspice_df
    display(tree_dict[branch]["auspice_df"])

NameError: name 'auspice_df' is not defined

---

# Plot Rates

In [ ]:
rates = [float(rate) for rate in list(tree_dict["1.ORI"]["df"]["rate"]) if rate != NO_DATA_CHAR]
print(sum(rates) / len(rates))
#sns.histplot(rates)


---

# Export

## Time Tree

In [ ]:
for branch in tree_dict:
    out_timetree = copy.deepcopy(tree_dict[branch]["tree"])

    metadata_to_comment(out_timetree, tree_dict[branch]["df"])    
    out_timetree_nex_path = os.path.join(tree_dir, branch + ".timetree.nex")
    Phylo.write(out_timetree, out_timetree_nex_path, "nexus")

    for c in out_timetree.find_clades():
        c.comment = None

    out_timetree_nwk_path = os.path.join(tree_dir, branch + ".timetree.nwk")
    Phylo.write(out_timetree, out_timetree_nwk_path, "newick")

## Augur

In [ ]:
for branch in tree_dict:
    
    augur_dict = augur_export(
        tree_path=None, 
        aln_path=None,  
        tree=tree_dict[branch]["tree"], 
        tree_df=tree_dict[branch]["auspice_df"], 
        color_keyword_exclude=["geometry"],
        type_convert = {
            "branch_number" : (lambda x : str(x))
        },
    )
    
    tree_dict[branch]["augur_dict"] = augur_dict
    
    first_taxa = list(augur_dict["nodes"].keys())[0]
    print(augur_dict["nodes"][first_taxa])

    out_path_augur_json = os.path.join(tree_dir, branch + "_augur.json" )
    utils.write_json(data=tree_dict[branch]["augur_dict"], file_name=out_path_augur_json, indent=JSON_INDENT)
    tree_dict[branch]["augur_json_path"] = out_path_augur_json

## Auspice

In [ ]:
for branch in tree_dict:
    
    # Store the color
    branch_major_color = colors_dict["branch_major"][branch]

    auspice_dict = auspice_export(
        tree=tree_dict[branch]["tree"],
        augur_json_paths=tree_dict[branch]["augur_json_path"], 
        auspice_config_path=auspice_config_path, 
        auspice_colors_path=out_path_colors,
        auspice_latlons_path=out_path_latlon, 
        )


    label_col = list(tree_dict[branch]["auspice_df"])
    print(label_col)

    # Recursively add branch attrs
    branch_attributes(
        tree_dict=auspice_dict["tree"], 
        sub_dict=auspice_dict["tree"], 
        df=tree_dict[branch]["auspice_df"],
        label_col=label_col,
        )
    
    
    # Last manual changes
    auspice_dict_copy = copy.deepcopy(auspice_dict)
    for i in range(0, len(auspice_dict_copy["meta"]["colorings"])):
        coloring = auspice_dict_copy["meta"]["colorings"][i]
        for key in coloring:
            # Node type as internal or terminal
            if coloring[key] == "node_type":
                auspice_dict["meta"]["colorings"][i]['scale'] = [['internal', '#FFFFFF'], ['terminal', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])
            # Confidence category
            if "conf_category" in coloring[key]:
                auspice_dict["meta"]["colorings"][i]['scale'] = [['LOW', '#FFFFFF'], ['HIGH', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])

    # Write outputs - For Local Rendering
    out_path_auspice_local_json = os.path.join(tree_dir, branch + ".json" )
    utils.write_json(data=auspice_dict, file_name=out_path_auspice_local_json, indent=JSON_INDENT, include_version=False)
    export_v2.validate_data_json(out_path_auspice_local_json)
    print("Validation successful for local JSON.\n")